# Luminar

In [ ]:
import warnings

warnings.filterwarnings("ignore", message=r".*Please note that with a fast tokenizer.*")
warnings.filterwarnings(
    "ignore",
    message=r".*Using the `WANDB_DISABLED` environment variable is deprecated.*",
)
warnings.filterwarnings(
    "ignore",
    message=r".*Was asked to gather along dimension \d+, but all input tensors were scalars.*",
)

In [ ]:
import gc
import json

import numpy as np
import torch
from datasets import Dataset, DatasetDict, load_dataset
from tqdm.auto import tqdm

from luminar.baselines.core import run_detector, run_detector_tokenized


## Baselines: Neural Network Models

### Data

In [ ]:
from pathlib import Path

from luminar.utils import get_matched_datasets

HF_TOKEN = (Path.home() / ".hf_token").read_text().strip()


def load_datasets(
    agent="gpt_4o_mini",
    other_agents="gemma2_9b",
    domains=(
        "blog_authorship_corpus",
        "student_essays",
        "cnn_news",
        "euro_court_cases",
        "house_of_commons",
        "arxiv_papers",
        "gutenberg_en",
        "bundestag",
        "spiegel_articles",
        # "gutenberg_de",
        "en",
        "de",
    ),
    num_proc: int = 32,
):
    datasets = {}
    for domain in tqdm(domains):
        datset_config_name = f"{domain}-fulltext"
        dataset_split_name = f"human+{agent}+{other_agents}"
        dataset: Dataset = (
            load_dataset(
                "liberi-luminaris/PrismAI",
                datset_config_name,
                split=dataset_split_name,
                token=HF_TOKEN,
            )  # type: ignore
            .rename_column("label", "labels")
            .filter(
                lambda text: len(text.strip()) > 0,
                input_columns=["text"],
                num_proc=num_proc,  # type: ignore
            )
        )
        datasets_matched, dataset_unmatched = get_matched_datasets(
            dataset, agent, num_proc=num_proc
        )
        datasets_matched["unmatched"] = dataset_unmatched
        datasets[domain] = datasets_matched
    return datasets


datasets = load_datasets()

### RADAR

In [ ]:
from luminar.baselines.radar import Radar


def run_radar():
    detector = Radar(device="cuda")
    try:
        return run_detector(detector, datasets, sigmoid=False)
    finally:
        detector.model.to("cpu")
        del detector
        gc.collect()
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()


scores_radar = run_radar()
print(json.dumps(scores_radar, indent=4))
with open("../logs/radar.json", "w") as fp:
    json.dump(scores_radar, fp, indent=4)

### Binoculars

In [ ]:
from luminar.baselines.binoculars import GLOBAL_BINOCULARS_THRESHOLD, Binoculars


def run_binoculars():
    detector = Binoculars("tiiuae/falcon-7b", "tiiuae/falcon-7b-instruct")
    try:
        return run_detector(
            detector,
            datasets,
            batch_size=16,
            threshold=GLOBAL_BINOCULARS_THRESHOLD,
            # lower scores indicate AI generated texts
            less_than=True,
        )
    finally:
        detector.observer_model.to("cpu")
        detector.performer_model.to("cpu")
        del detector
        gc.collect()
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()


scores_binoculars = run_binoculars()
print(json.dumps(scores_binoculars, indent=4))
with open("../logs/binoculars.json", "w") as fp:
    json.dump(scores_binoculars, fp, indent=4)

### E5-Small LoRA


In [ ]:
from luminar.baselines.trainable.e5_lora import E5Lora


def run_e5_small_lora():
    detector = E5Lora(device="cuda:0")
    try:
        return run_detector(detector, datasets)
    finally:
        detector.model.to("cpu")
        del detector
        gc.collect()
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()


scores_e5 = run_e5_small_lora()

print(json.dumps(scores_e5, indent=4))
with open("../logs/e5-small-lora.json", "w") as fp:
    json.dump(scores_e5, fp, indent=4)

## Evaluation

In [ ]:
domains = [
    "Web Blogs",
    "Essays",
    "CNN",
    "ECHR",
    "HoC",
    "arXiv",
    "Gutenberg$_{en}$",
    "Bundestag$_{de}$",
    "Spiegel$_{de}$",
    "Gutenberg$_{de}$",
    "All$_{en}$",
    "All$_{de}$",
]

name_map = {
    "blog_authorship_corpus": "Web Blogs",
    "student_essays": "Essays",
    "cnn_news": "CNN",
    "euro_court_cases": "ECHR",
    "house_of_commons": "HoC",
    "arxiv_papers": "arXiv",
    "gutenberg_en": "Gutenberg$_{en}$",
    "bundestag": "Bundestag$_{de}$",
    "spiegel_articles": "Spiegel$_{de}$",
    # "gutenberg_de": "Gutenberg$_{de}$",
    "en": "All$_{en}$",
    "de": "All$_{de}$",
}

In [ ]:
import json
from collections import defaultdict
from pathlib import Path

import pandas as pd

results = defaultdict(dict)
for logs_path in Path("../logs/").iterdir():
    if logs_path.suffix == ".json":
        with logs_path.open("r") as fp:
            data = json.load(fp)
        model_name = logs_path.stem
        for domain, scores in data.items():
            results[name_map[domain]].update(
                {
                    model_name + "_f1_score": scores["f1_score_fpr"],
                    # model_name + "_accuracy": scores["accuracy_fpr"],
                    model_name + "_tpr": scores["tpr_fpr"],
                    model_name + "_roc_auc": scores["roc_auc_fpr"],
                }
            )

model_name = "roberta-base-ft"
for domain, name in name_map.items():
    logs_path = Path("../logs/roberta-ft/roberta-base/") / (domain + ".json")
    with (logs_path).open("r") as fp:
        data = json.load(fp)
    results[name].update(
        {
            model_name + "_f1_score": data[domain]["f1_score_fpr"],
            # model_name + "_accuracy": data[domain]["accuracy_fpr"],
            model_name + "_tpr": data[domain]["tpr_fpr"],
            model_name + "_roc_auc": data[domain]["roc_auc_fpr"],
        }
    )

metric_df = (
    pd.DataFrame([{"domain": domain} | dd for domain, dd in results.items()])
    .set_index("domain")
    .sort_index(key=lambda x: list(map(domains.index, x)))
)
print(metric_df.to_latex(float_format="%.3f", index=True))
metric_df

In [ ]:
import json
from collections import defaultdict
from pathlib import Path

import pandas as pd

results = defaultdict(dict)
for logs_path in Path("../logs/").iterdir():
    if logs_path.suffix == ".json":
        with logs_path.open("r") as fp:
            data = json.load(fp)
        model_name = logs_path.stem
        for domain, scores in data.items():
            results[name_map[domain]].update(
                {
                    model_name + "_f1_score": scores["f1_score"],
                    model_name + "_accuracy": scores["accuracy"],
                    model_name + "_roc_auc": scores["roc_auc"],
                }
            )

model_name = "roberta-base-ft"
for domain, name in name_map.items():
    logs_path = Path("../logs/roberta-ft/roberta-base/") / (domain + ".json")
    with (logs_path).open("r") as fp:
        data = json.load(fp)
    results[name].update(
        {
            model_name + "_f1_score": data[domain]["f1_score"],
            model_name + "_accuracy": data[domain]["accuracy"],
            model_name + "_roc_auc": data[domain]["roc_auc"],
        }
    )

metric_df = (
    pd.DataFrame([{"domain": domain} | dd for domain, dd in results.items()])
    .set_index("domain")
    .sort_index(key=lambda x: list(map(domains.index, x)))
)
print(metric_df.to_latex(float_format="%.3f", index=True))
metric_df

In [ ]:
import json
from collections import defaultdict
from pathlib import Path

import pandas as pd

results = defaultdict(dict)
for logs_path in Path("../logs/").iterdir():
    if logs_path.suffix == ".json":
        with logs_path.open("r") as fp:
            data = json.load(fp)
        model_name = logs_path.stem
        for domain, scores in data.items():
            results[name_map[domain]].update(
                {
                    model_name + "_f1_score": scores["f1_score_median"],
                    model_name + "_accuracy": scores["accuracy_median"],
                    model_name + "_roc_auc": scores["roc_auc_median"],
                }
            )

model_name = "roberta-base-ft"
for domain, name in name_map.items():
    logs_path = Path("../logs/roberta-ft/roberta-base/") / (domain + ".json")
    with (logs_path).open("r") as fp:
        data = json.load(fp)
    results[name].update(
        {
            model_name + "_f1_score": data[domain]["f1_score_median"],
            model_name + "_accuracy": data[domain]["accuracy_median"],
            model_name + "_roc_auc": data[domain]["roc_auc_median"],
        }
    )

metric_df = (
    pd.DataFrame([{"domain": domain} | dd for domain, dd in results.items()])
    .set_index("domain")
    .sort_index(key=lambda x: list(map(domains.index, x)))
)
print(metric_df.to_latex(float_format="%.3f", index=True))
metric_df

In [ ]:
import json
from collections import defaultdict
from pathlib import Path

import pandas as pd

results = defaultdict(dict)
for logs_path in Path("../logs/").iterdir():
    if logs_path.suffix == ".json":
        with logs_path.open("r") as fp:
            data = json.load(fp)
        model_name = logs_path.stem
        for domain, scores in data.items():
            results[name_map[domain]].update(
                {
                    model_name + "_f1_score": scores["f1_score_mean"],
                    model_name + "_accuracy": scores["accuracy_mean"],
                    model_name + "_roc_auc": scores["roc_auc_mean"],
                }
            )

model_name = "roberta-base-ft"
for domain, name in name_map.items():
    logs_path = Path("../logs/roberta-ft/roberta-base/") / (domain + ".json")
    with (logs_path).open("r") as fp:
        data = json.load(fp)
    results[name].update(
        {
            model_name + "_f1_score": data[domain]["f1_score_mean"],
            model_name + "_accuracy": data[domain]["accuracy_mean"],
            model_name + "_roc_auc": data[domain]["roc_auc_mean"],
        }
    )

metric_df = (
    pd.DataFrame([{"domain": domain} | dd for domain, dd in results.items()])
    .set_index("domain")
    .sort_index(key=lambda x: list(map(domains.index, x)))
)
print(metric_df.to_latex(float_format="%.3f", index=True))
metric_df

In [ ]:
results = defaultdict(dict)

model_name = "roberta-ft"
for domain, name in name_map.items():
    logs_path = Path("../logs/chatgpt-detector-roberta/") / (domain + ".json")
    with (logs_path).open("r") as fp:
        data = json.load(fp)
    results[name].update(
        {
            model_name + "_f1": data[domain]["f1"],
            # model_name + "_accuracy": data[domain]["accuracy"],
            model_name + "_auroc": data[domain]["auroc"],
        }
    )

metric_df = (
    pd.DataFrame([{"domain": domain} | dd for domain, dd in results.items()])
    .set_index("domain")
    .sort_index(key=lambda x: list(map(domains.index, x)))
)
print(metric_df.to_latex(float_format="%.3f", index=True))
metric_df